In [ ]:
!pip install peft
!pip install datasets
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, get_scheduler, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.metrics import roc_auc_score
from peft import LoraConfig, get_peft_model, TaskType
from torch.utils.data import DataLoader, Dataset
import wandb
import re
import time
import pandas as pd

# Initialize wandb
wandb.init(project="LoRA_Ablation_Study", entity="siyinggu-nyu")

# Helper function to clean text
def clean_text(text):
    if text is None:
        return ""
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.lower()  # Convert to lowercase
    return text.strip()

# Function to clean the entire dataset
def clean_dataset(dataset):
    dataset = dataset.map(lambda x: {'text': clean_text(x['text'])})
    return dataset

# Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Helper function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(logits, dim=-1)
    accuracy = (preds == labels).sum().item() / labels.shape[0]
    return {'accuracy': accuracy}

# Function to get the number of trainable parameters
def get_model_size(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Function to get memory usage
def get_memory_usage():
    return torch.cuda.memory_allocated()

# Load and clean dataset
dataset = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")
dataset = clean_dataset(dataset)

# Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
test_texts = dataset['test']['text']
test_labels = dataset['test']['label']

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Function to perform ablation study
def run_ablation_study(target_layers, rank, alpha):
    model_name = "distilbert-base-uncased"
    lora_config = LoraConfig(
        r=rank,
        lora_alpha=alpha,
        lora_dropout=0.05,
        target_modules=target_layers,
        bias='none',
        task_type=TaskType.SEQ_CLS
    )

    model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=3)
    peft_model = get_peft_model(model, lora_config)

    training_args = TrainingArguments(
        output_dir='./results',
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        weight_decay=0.01,
        evaluation_strategy='steps',
        eval_steps=10,
        logging_dir='./logs',
        logging_steps=10,
        seed=42,
        fp16=True, # Only use with GPU
        report_to="wandb"
    )

    trainer = Trainer(
        model=peft_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    start_time = time.time()
    trainer.train()
    end_time = time.time()

    eval_results = trainer.evaluate()
    trainable_params = get_model_size(peft_model)
    memory_usage = get_memory_usage()
    accuracy = eval_results['eval_accuracy']

    return {
        'target_layers': target_layers,
        'rank': rank,
        'alpha': alpha,
        'eval_results': eval_results,
        'training_time': end_time - start_time,
        'accuracy': accuracy,
        'trainable_params': trainable_params,
        'memory_usage': memory_usage
    }

# Define ablation study parameters
target_layers_list = [["q_lin"], ["q_lin", "k_lin"], ["q_lin", "k_lin", "v_lin"]]
rank_list = [4, 8, 16] #
alpha_list = [2, 16, 32] #big alpha/rand ratio

# Run ablation study
results = []
for target_layers in target_layers_list:
    for rank in rank_list:
        for alpha in alpha_list:
            result = run_ablation_study(target_layers, rank, alpha)
            print(result)
            results.append(result)
            wandb.log(result)

# Save results
df_results = pd.DataFrame(results)
df_results.to_csv('ablation_study_results.csv', index=False)
print(df_results)


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
